## SVM进行二分类手写数字集的分类(标签为0和标签不为0)
### 准确率98.03%

In [29]:
import numpy as np
import pandas as pd
import plotly.express as px
from tqdm import tqdm_notebook as tqdm

In [30]:
trainData = np.array(pd.read_csv("trainData.csv").iloc[:,1:])
trainlabel = np.array(pd.read_csv("trainData.csv").iloc[:,0])
testData = np.array(pd.read_csv("testData.csv").iloc[:,1:])
testlabel = np.array(pd.read_csv("testData.csv").iloc[:,0])
trainlabel[trainlabel>0]=1;trainlabel[trainlabel==0]=-1
testlabel[testlabel>0]=1;testlabel[testlabel==0]=-1

In [31]:
trainData.shape,testData.shape,trainlabel.shape,testlabel.shape

((60000, 784), (10000, 784), (60000,), (10000,))

In [32]:
class SVM():
    def __init__(self,trainData,trainlabel,C=100,iters=300,sigma=10,toler=0.001):
        self.num = trainData.shape[0]#获得总样本值
        self.sigma = sigma
        self.trainData=trainData
        self.trainlabel=trainlabel   
        self.Kij = self.calKernel(trainData)
        self.AlphaJ = [0]*trainData.shape[0]### 初始化所有的ai点
        self.bias = 0
        self.C = C
        self.iters = iters
        self.suportVector = []
        self.toler=toler
        
    def calGuassion(self,xi,trainData):
        Ki = np.exp(-np.linalg.norm([xi-trainData],axis=2,ord=2)/(2*self.sigma**2))
        return Ki
    
    def calKernel(self,trainData): #计算高斯核
        K = np.zeros((trainData.shape[0],trainData.shape[0]),np.float32)
        with tqdm(total=trainData.shape[0]) as bar:
            for i in range(trainData.shape[0]):
                xi=trainData[i].reshape(1,-1) ###784列的向量
                Ki = self.calGuassion(xi,self.trainData)
                K[i]=Ki
                bar.update(1)
                bar.set_postfix({"kernel":Ki})
        return K
    
    def calGxi(self,index):### 计算gxi
        AjYj=np.array(self.AlphaJ)*self.trainlabel
        gxi = np.dot(AjYj,self.Kij[index])+self.bias
        return gxi
    
    def calGuassKernel(self,x1,x2):
        return  np.exp(-np.linalg.norm([x1-x2],axis=2,ord=2)/(2*self.sigma**2))
    
    def isSatisfyKKT(self,index): ### 判断该优化点是否满足KKT条件
        yi = self.trainlabel[index]
        gxi = self.calGxi(index)
        ai = self.AlphaJ[index]
        if -self.toler<=ai<=self.toler and yi*gxi>=1:
            return True
        elif self.toler<ai<self.C and yi*gxi==1:
            return True
        elif self.C<=ai<=self.C+self.toler and yi*gxi<=1:
            return True
        return False
    
    def calEi(self,index):#
        gxi=self.calGxi(index)
        yi = self.trainlabel[index]
        return gxi-yi
        
    def findE2(self,E1):
        Ei=np.zeros(self.trainData.shape[0])
        for i in range(self.trainData.shape[0]):
            Ei[i]=self.calEi(i) ### 计算每个Ei值
        alpha2_index=np.argmax(np.abs(E1-Ei))
        E2= Ei[alpha2_index]
        return alpha2_index,E2
        
    def train(self):
        ### smo序列优化算法，
        #首先外循环遍历第一个需要优化的样本
        step = 0
        paramtersChange= -1
        while step<=self.iters and paramtersChange!=0:
            paramtersChange=0
            step+=1
            with tqdm(total=self.trainData.shape[0]) as bar:
                for i in range(self.trainData.shape[0]):
                    bar.update(1)
                # 寻找违反kkt条件的支持向量
                    if self.isSatisfyKKT(i)!=True: ##不满足KKT条件即为第一个需要优化的点,书7.4.2
                        alpha1_index = i
                        E1=self.calEi(i)
                        #寻找到第一个优化点后，在寻找第二个优化点时，需要是的abs(E1.E2)的值尽可能的大
                        alpha2_index,E2 = self.findE2(E1)## 未经过剪裁的值

                        ###更新alpha1 alpha2
                        eta = self.Kij[alpha1_index][alpha1_index] + self.Kij[alpha2_index][alpha2_index] - 2*self.Kij[alpha1_index][alpha2_index]  
                        alpha2_unc = self.AlphaJ[alpha2_index]+self.trainlabel[alpha2_index]*(E1-E2)/eta

                        ## 计算取值范围
                        y1=self.trainlabel[i]
                        y2=self.trainlabel[alpha2_index]
                        a1,a2 = self.AlphaJ[alpha1_index],self.AlphaJ[alpha2_index]

                        L = max([0,a2-a1]) if y1!=y2 else max([0,a2+a1-self.C])
                        H = min([self.C,a2-a1+self.C]) if y1!=y2 else min([self.C,a2+a1])
                        if L==H:
                            continue
                        else:
                            if alpha2_unc>H: alpha2_new=H
                            elif L<=alpha2_unc<=H: alpha2_new = alpha2_unc
                            elif alpha2_unc<L: alpha2_new =L
                        alpha1_new = a1 + y1*y2*(a2-alpha2_new)
                        ## 更新bias
                        b1new = -E1 -y1*self.Kij[alpha1_index][alpha1_index]*(alpha1_new-a1)-y2*self.Kij[alpha2_index][alpha1_index]*(alpha2_new-a2)+self.bias
                        b2new = -E2 -y1*self.Kij[alpha1_index][alpha2_index]*(alpha1_new-a1)-y2*self.Kij[alpha2_index][alpha2_index]*(alpha2_new-a2)+self.bias
                        
                        if 0<alpha1_new<self.C:
                                bnew = b1new
                        elif 0<alpha2_new<self.C:
                                bnew = b2new
                        else:
                                bnew = (b1new+b2new)/2
                        self.b = bnew
                        self.AlphaJ[alpha1_index] = alpha1_new
                        self.AlphaJ[alpha2_index] = alpha2_new
                        if alpha2_new -a2>=0.0001:
                            paramtersChange+=1
                    if step<=3:
                        bar.set_postfix({"a":self.AlphaJ,"parameterChanges":paramtersChange})
                    else:
                        bar.set_postfix({"parameterChanges":paramtersChange})
        for i,alpha in enumerate(self.AlphaJ):
            if alpha>0:
                self.suportVector.append(i)
    
    
    def predict(self, x):
        result = 0
        for i in self.suportVector:
            #遍历所有支持向量，计算求和式
            #如果是非支持向量，求和子式必为0，没有必须进行计算
            #这也是为什么在SVM最后只有支持向量起作用
            #------------------
            #先单独将核函数计算出来
            tmp = self.calGuassKernel(self.trainData[i].reshape(1,-1),x.reshape(1,-1))
            #对每一项子式进行求和，最终计算得到求和项的值
            result += self.AlphaJ[i] * self.trainlabel[i] * tmp
        #求和项计算结束后加上偏置b
        result += self.b
        #使用sign函数返回预测结果
        return np.sign(result)  ### sign函数,输出为1或-1

In [33]:
svm = SVM(trainData[:1000],trainlabel[:1000]) #计算高斯kernel矩阵

C:\Users\YGW\AppData\Local\Temp/ipykernel_13364/761360924.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  with tqdm(total=trainData.shape[0]) as bar:


  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
svm.train()

C:\Users\YGW\AppData\Local\Temp/ipykernel_13364/761360924.py:71: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  with tqdm(total=self.trainData.shape[0]) as bar:


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
prelabel =np.zeros_like(testlabel)

In [ ]:
for i in tqdm(range(prelabel.shape[0])):
    prelabel[i] = svm.predict(testData[i])
acc = np.sum(prelabel==testlabel)/prelabel.shape[0] *100

In [ ]:
print(f"\033[94m准确率:{acc}%")